In [ ]:
import os
%pwd

In [2]:
os.chdir("../")

In [ ]:
%pwd

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from dialogue_rag_chatbot.constants import *
from dialogue_rag_chatbot.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import py7zr
from dialogue_rag_chatbot.logging import logger
from dialogue_rag_chatbot.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with py7zr.SevenZipFile(self.config.local_data_file, mode="r") as z:
            z.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-24 17:46:42,422: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-24 17:46:42,426: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 17:46:42,429: INFO: common: created directory at: artifacts]
[2025-09-24 17:46:42,431: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-24 17:46:45,781: INFO: 3505561092: artifacts/data_ingestion/data.7z download! with following info: 
Connection: close
Content-Length: 2944100
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/octet-stream
ETag: "4450eb63089f56212a41409ae9bcb15b3fb051149cf8d4b895f422d5c86b831c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3EDE:3B1212:1867C6:41AE2F:68D3BE03
Accept-Ranges: bytes
Date: Wed, 24 Sep 2025 09:46:44 GMT
Via: 1.1 varnish
X-Served-By: cache-mnl9728-MNL
X-C